## Data Cleanup

### Imports & EC3 Python Wrapper Setup

In [12]:
import os
import pandas as pd
import json

### Define Cleaning Functions

In [13]:
def remove_outliers(df, col_names):
    """
    Remove extreme outliers based on IQR method
    """
    q1 = df[col_names].quantile(0.25)
    q3 = df[col_names].quantile(0.75)
    iqr = q3 - q1

    df = df[~((df[col_names] < (q1 - 1.5 * iqr)) |(df[col_names] > (q3 + 1.5 * iqr))).any(axis=1)]

    return df.copy()

### Load JSON

In [14]:
# Navigate into the 01_raw_data folder to load the data
os.chdir(os.path.dirname(os.getcwd()))
os.chdir('01_raw_data')

with open('epddata_4000psi_test.json', 'r') as f:
    epd_list_4000 = json.load(f)

### Store just what is needed for now

In [15]:
def convert_epd_list(epd_list):
    filt_epd_list = []
    for epd in epd_list:
        new_dict = {}
        new_dict['id'] = epd['id']
        new_dict['date_of_issue'] = epd['date_of_issue']
        new_dict['compressive_strength'] = epd.get('concrete_compressive_strength_28d')
        new_dict['gwp'] = epd.get('gwp')
        new_dict['gwp_per_category_declared_unit'] = epd.get('gwp_per_category_declared_unit')
        new_dict['plant_geography'] = epd.get('plant_geography')
        new_dict['cementitious'] = epd.get('cementitious')
        new_dict['plant_country'] =  new_dict['plant_geography'][0][0:2]
        new_dict['plant_subdiv'] = new_dict['plant_geography'][0][-2:]

        filt_epd_list.append(new_dict)

    return filt_epd_list

In [16]:
filt_epd_list_4000 = convert_epd_list(epd_list_4000)

### Cleaning the Data

In [17]:
# convert to dataframe and drop rows missing critical values
df_4ksi = pd.DataFrame(filt_epd_list_4000)
df_4ksi.dropna(subset=['compressive_strength', 'gwp', 'plant_country'], inplace=True)
df_4ksi = df_4ksi[df_4ksi['plant_country'] == 'US'] #extra filtering for plant country (may not be needed for all mixes)

In [18]:
#add a column for the gwp formatted as a float converted to kgCO2e per cubic yard
df_4ksi['gwp_val'] = (
    df_4ksi['gwp_per_category_declared_unit']
    .str.extract(r'(\d+\.?\d*)')
    .astype(float)
    .round(1) * 0.764555
)

In [ ]:
# remove epd if gwp value is an outlier
df_4ksi = remove_outliers(df_4ksi, ['gwp_val'])

#add a column for the date formatted as a datetime object
df_4ksi['date_formatted'] = pd.to_datetime(df_4ksi['date_of_issue'], format='%Y-%m-%d')

df_4ksi = df_4ksi[df_4ksi['date_formatted'].dt.year >= 2019] #only keep EPDs from 2019 and later

len(df_4ksi) #check length of dataset after cleaning steps

13

In [21]:
df_4ksi

,id,date_of_issue,compressive_strength,gwp,gwp_per_category_declared_unit,plant_geography,cementitious,plant_country,plant_subdiv,gwp_val,date_formatted
20,4720d61372684e1caf6313e5faec3981,2019-02-11,27.6 MPa,325 kgCO2e,325 kgCO2e,[US-NC],{'fly_ash': 0.0},US,NC,248.480375,2019-02-11
22,d988699badc94d90b4f2311c99758626,2020-03-25,27.6 MPa,317 kgCO2e,317 kgCO2e,[US-GA],{'fly_ash': 0.0},US,GA,242.363935,2020-03-25
23,373379abf0444d90b9c7756ba3172cda,2020-03-25,27.6 MPa,338 kgCO2e,338 kgCO2e,[US-GA],{'fly_ash': 0.0},US,GA,258.419590,2020-03-25
24,1910cbb6ee184a25b6db81bc3833c205,2020-04-20,27.6 MPa,334 kgCO2e,334 kgCO2e,[US-NC],{'fly_ash': 0.0},US,NC,255.361370,2020-04-20
25,ef00d9e8277d4897b665235b521c24ea,2020-04-20,27.6 MPa,322 kgCO2e,322 kgCO2e,[US-NC],{'fly_ash': 0.0},US,NC,246.186710,2020-04-20
26,7654086c99024cc3958ee9d633b884c2,2020-04-21,27.6 MPa,322 kgCO2e,322 kgCO2e,[US-NC],{'fly_ash': 0.0},US,NC,246.186710,2020-04-21
27,25d74a0ca99747a5877a1164a6a92ea3,2020-04-21,27.6 MPa,322 kgCO2e,322 kgCO2e,[US-NC],{'fly_ash': 0.0},US,NC,246.186710,2020-04-21
28,db4179c53f2047bbb6a0f7188125cf5d,2020-06-04,27.6 MPa,379 kgCO2e,379 kgCO2e,[US-GA],{'fly_ash': 0.0},US,GA,289.766345,2020-06-04
29,375689008801466db143dffed58c4de1,2020-06-09,27.6 MPa,297 kgCO2e,297 kgCO2e,[US-GA],{'fly_ash': 0.0},US,GA,227.072835,2020-06-09
30,29eb45928c13403f95521a6085ab8e07,2020-06-09,27.6 MPa,380 kgCO2e,380 kgCO2e,[US-GA],{'fly_ash': 0.0},US,GA,290.530900,2020-06-09
